In [ ]:
# 📦 1. Importar librerías principales
import os
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 📁 2. Configurar paths automáticamente según entorno (Colab o local)
# Ruta base = carpeta ChurnNN
BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Agregar src al path para importar módulos
SRC_PATH = os.path.join(BASE_PATH, "src")
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

# 📁 3. Cargar y explorar el dataset
data_path = os.path.join(BASE_PATH, "data", "churn_dataset.csv")
df = pd.read_csv(data_path)
df.head()

In [ ]:
# 🔍 4. Separar variables predictoras y variable objetivo
X = df.drop("churn", axis=1)
y = df["churn"]

# 🧼 5. Escalar los datos y dividir en train/test
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [ ]:
# 🧠 6. Cargar configuración y modelo
import yaml
from model import build_model
from utils import plot_metrics, show_confusion_matrix

# Cargar parámetros desde config.yaml
config_path = os.path.join(SRC_PATH, "config.yaml")
with open(config_path) as f:
    config = yaml.safe_load(f)

# Construir modelo
model = build_model(config)

In [ ]:
# 🚂 7. Entrenar el modelo
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=config['training']['epochs'],
    batch_size=config['training']['batch_size']
)

In [ ]:
# 📉 8. Visualizar métricas de entrenamiento
plot_metrics(history)

In [ ]:
# 🧪 9. Evaluar el modelo en test
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

show_confusion_matrix(y_test, y_pred_classes)